# Berke Lab Spike Sorting !

In [1]:
import datajoint as dj
import numpy as np

import spyglass.common as sgc
import spyglass.spikesorting.v1 as sgs
import spyglass.position as sgp
import spyglass.data_import as sgi
from spyglass.common import Session, Nwbfile
from spyglass.utils.nwb_helper_fn import get_nwb_file

# Make sure the session exists
#nwb_file_name = "IM-1594_20230726_.nwb"
nwb_file_name =  "IM-1478_20220726_.nwb"

# Fetch file create date and source version to make sure it's up to date
nwb_file_abspath = Nwbfile.get_abs_path(nwb_file_name)
nwbf = get_nwb_file(nwb_file_abspath)
print(f"File created on {nwbf.file_create_date[0].strftime('%m/%d/%Y %H:%M:%S')}")
print(f"Source script version {nwbf.source_script}")

[2025-11-06 15:13:09,628][INFO]: DataJoint 0.14.4 connected to scrater@lmf-db.cin.ucsf.edu:3306


File created on 10/29/2025 18:00:06
Source script version jdb_to_nwb 2.1.2.dev2+g12a9b6eed.d20251030


Take a quick look at the parameters we are using!

In [2]:
## Parameters

# Preprocessing
preproc_param_name = "franklab_tetrode_hippocampus"
artifact_param_name = "ampl_1000_z_30_prop_075_1ms"
# Sorting
sorter = "mountainsort4"
sorter_param_name = "franklab_tetrode_hippocampus_30KHz"
# Curation
waveform_param_name = "default_not_whitened"
metric_param_name = "franklab_default"
metric_curation_param_name = "default"
# Decoding
#unit_filter_params_name = "default_exclusion"
# waveform_features_group_name = position_group_name = "1000uV_artifact_100uV_detection_custom_channels"
# trodes_pos_params_name = "berke_double_led_500" # led1_is_front = 0, upsampling_sampling_rate = 500
# decoding_param_name = "contfrag_clusterless_252ch_4chunks_2000block_3bin"
#from spyglass.spikesorting.analysis.v1.group import UnitSelectionParams

# Optionally print them all!
review_params = True
if review_params:
    display((sgs.SpikeSortingPreprocessingParameters() & {"preproc_param_name": preproc_param_name}).fetch1())

    display((sgs.ArtifactDetectionParameters() & {"artifact_param_name": artifact_param_name}).fetch1("artifact_params"))

    display((sgs.SpikeSorterParameters() & {"sorter": sorter, "sorter_param_name": sorter_param_name}).fetch1())

    display((sgs.WaveformParameters() & {"waveform_param_name": waveform_param_name}).fetch1())

    display((sgs.MetricParameters() & {"metric_param_name": metric_param_name}).fetch1())

    display((sgs.MetricCurationParameters() & {"metric_curation_param_name": metric_curation_param_name}).fetch1())

    # display((sgp.v1.TrodesPosParams() & {"trodes_pos_params_name": trodes_pos_params_name}).fetch1("params"))

    #display((UnitSelectionParams() & {"unit_filter_params_name": unit_filter_params_name}).fetch1())

    # display((DecodingParameters & {"decoding_param_name": decoding_param_name}).fetch('decoding_kwargs'))

    # display((DecodingParameters & {"decoding_param_name": decoding_param_name}).fetch('decoding_params'))


{'preproc_param_name': 'franklab_tetrode_hippocampus',
 'preproc_params': {'frequency_min': 600,
  'frequency_max': 6000,
  'margin_ms': 5,
  'seed': 0,
  'min_segment_length': 0.0015}}

{'zscore_thresh': 30,
 'amplitude_thresh_uV': 1000,
 'proportion_above_thresh': 0.75,
 'removal_window_ms': 1,
 'chunk_duration': '10s',
 'n_jobs': 4,
 'progress_bar': 'True'}

{'sorter': 'mountainsort4',
 'sorter_param_name': 'franklab_tetrode_hippocampus_30KHz',
 'sorter_params': {'detect_sign': -1,
  'adjacency_radius': 100,
  'freq_min': 600,
  'freq_max': 6000,
  'filter': False,
  'whiten': True,
  'num_workers': 1,
  'clip_size': 40,
  'detect_threshold': 3,
  'detect_interval': 10}}

{'waveform_param_name': 'default_not_whitened',
 'waveform_params': {'ms_before': 0.5,
  'ms_after': 0.5,
  'max_spikes_per_unit': 5000,
  'n_jobs': 5,
  'total_memory': '5G',
  'whiten': False}}

{'metric_param_name': 'franklab_default',
 'metric_params': {'snr': {'peak_sign': 'neg',
   'random_chunk_kwargs_dict': {'num_chunks_per_segment': 20,
    'chunk_size': 10000,
    'seed': 0}},
  'isi_violation': {'isi_threshold_ms': 1.5, 'min_isi_ms': 0.0},
  'nn_isolation': {'max_spikes': 1000,
   'min_spikes': 10,
   'n_neighbors': 5,
   'n_components': 7,
   'radius_um': 100,
   'seed': 0},
  'nn_noise_overlap': {'max_spikes': 1000,
   'min_spikes': 10,
   'n_neighbors': 5,
   'n_components': 7,
   'radius_um': 100,
   'seed': 0},
  'peak_channel': {'peak_sign': 'neg'},
  'num_spikes': {}}}

{'metric_curation_param_name': 'default',
 'label_params': {'nn_noise_overlap': ['>', 0.1, ['noise', 'reject']]},
 'merge_params': {}}

# First check out all existing entries for this nwb

Helpful if we're halfway through running this.

In [3]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput

# Find all entries for this nwb in SortGroup
print(f"Entries for {nwb_file_name} in sgs.SortGroup")
display((sgs.SortGroup & {"nwb_file_name": nwb_file_name}))

# Find all entries for this nwb in SpikeSortingRecordingSelection
print(f"Entries for {nwb_file_name} in sgs.SpikeSortingRecordingSelection (one per SortGroup, or more if we are trying multiple preprocessing params)")
display(sgs.SpikeSortingRecordingSelection() & {'nwb_file_name': nwb_file_name})

# Fetch the recording ids (there is one for each sort group)
recording_ids = (sgs.SpikeSortingRecordingSelection() & {'nwb_file_name': nwb_file_name}).fetch("KEY")

# Use the recording_ids to get the preprocessed recording for each
print(f"Entries for {nwb_file_name} in sgs.SpikeSortingRecording (one per SortGroup)")
display(sgs.SpikeSortingRecording() & recording_ids)

# Get the detected artifact times for this recording
print(f"Entries for {nwb_file_name} in sgs.ArtifactDetectionSelection (one per recording_id)")
display(sgs.ArtifactDetectionSelection() & recording_ids)
artifact_ids = (sgs.ArtifactDetectionSelection() & recording_ids).fetch("KEY")

print(f"Entries for {nwb_file_name} in sgs.ArtifactDetection (one per recording_id)")
display(sgs.ArtifactDetection() & artifact_ids)

print(f"Entries for {nwb_file_name} in sgs.SpikeSortingSelection")
display(sgs.SpikeSortingSelection() & {'nwb_file_name': nwb_file_name})
sorting_ids = (sgs.SpikeSortingSelection() & {'nwb_file_name': nwb_file_name}).fetch("KEY")

print(f"Entries for {nwb_file_name} in sgs.SpikeSorting")
display(sgs.SpikeSorting() & sorting_ids)

print(f"Entries for {nwb_file_name} in sgs.CurationV1")
display(sgs.CurationV1() & sorting_ids)

print(f"Entries for {nwb_file_name} in sgs.MetricCurationSelection")
display(sgs.MetricCurationSelection() & sorting_ids)

print(f"Entries for {nwb_file_name} in sgs.SpikeSortingOutput")
merge_ids = SpikeSortingOutput().get_restricted_merge_ids(key={"nwb_file_name": nwb_file_name}, as_dict=True)
display(SpikeSortingOutput & merge_ids)

Entries for IM-1478_20220726_.nwb in sgs.SortGroup


nwb_file_name name of the NWB file,sort_group_id,sort_reference_electrode_id the electrode to use for referencing
IM-1478_20220726_.nwb,0,-1
IM-1478_20220726_.nwb,1,-1
IM-1478_20220726_.nwb,2,-1
IM-1478_20220726_.nwb,3,-1
IM-1478_20220726_.nwb,4,-1
IM-1478_20220726_.nwb,5,-1
IM-1478_20220726_.nwb,6,-1
IM-1478_20220726_.nwb,7,-1
IM-1478_20220726_.nwb,8,-1
IM-1478_20220726_.nwb,9,-1


Entries for IM-1478_20220726_.nwb in sgs.SpikeSortingRecordingSelection (one per SortGroup, or more if we are trying multiple preprocessing params)


recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name
0262eaeb-75c2-4285-9df2-b980b73da491,IM-1478_20220726_.nwb,48,00_r1,default,Berke Lab and friends
03524c38-9984-4d34-a7cd-f2d709ec56d6,IM-1478_20220726_.nwb,20,00_r1,default,Berke Lab and friends
06498a5e-5fc1-4d4c-914c-08ce27948b25,IM-1478_20220726_.nwb,41,00_r1,default,Berke Lab and friends
0867803c-2e78-4ac6-b4e2-6b085ca255ed,IM-1478_20220726_.nwb,4,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
0970abac-28bb-4eff-b5db-6e56a35bd938,IM-1478_20220726_.nwb,57,00_r1,default,Berke Lab and friends
0d2e1d88-a0a1-4b08-8108-1003a19914e9,IM-1478_20220726_.nwb,7,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
12616d5f-7f61-4f6a-95fe-b5c9fa1e1f6d,IM-1478_20220726_.nwb,47,00_r1,default,Berke Lab and friends
13b47c72-6b7d-462c-a0dc-24c3cf1f5ce9,IM-1478_20220726_.nwb,18,00_r1,default,Berke Lab and friends
179d1670-5dc9-4e8e-993c-22c3357da294,IM-1478_20220726_.nwb,2,00_r1,default,Berke Lab and friends
1aeaefa8-1f5e-4ec7-b603-4f0d62d4325b,IM-1478_20220726_.nwb,0,00_r1,default,Berke Lab and friends


Entries for IM-1478_20220726_.nwb in sgs.SpikeSortingRecording (one per SortGroup)


...

Entries for IM-1478_20220726_.nwb in sgs.MetricCurationSelection


metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
0bc55909-59fd-405a-8a94-b41d94a27804,373f4eb7-9411-41bf-ad85-16bbaccb3dfa,0,default_not_whitened,franklab_default,default
0eaaadfd-5e07-45cd-9d46-238bca751b54,a28382e1-eb6d-4411-95e3-c8c7a6fe2798,0,default_not_whitened,franklab_default,default
1d638c7f-65c8-4560-90b9-27d423148b52,c9da7594-c6ee-43b4-9a1e-7c9bc21906f4,0,default_not_whitened,franklab_default,default
227aa0f6-46c3-4a8c-85a6-b8245306ff70,a12af572-ee69-4dac-ae34-ec7252e2a92d,0,default_not_whitened,franklab_default,default
22c897d8-0ad8-427a-9729-48149b22e918,b7c8c657-05ef-4b15-9a0f-a00218d5163c,0,default_not_whitened,franklab_default,default
2908fb7b-ced5-4450-ac94-b86d79b8f700,b563f306-5d29-4279-8d6e-c97cbcc407e0,0,default_not_whitened,franklab_default,default
2b139787-a9f5-4bb3-8b7a-b60504a4f2f2,aa35a78e-3f18-4e47-bf62-4bd47ed2f79f,0,default_not_whitened,franklab_default,default
2ee3c57f-055b-4d79-be10-d16de8eddafe,7617faba-e28c-4846-b04e-3dbdac4f4540,0,default_not_whitened,franklab_default,default
365aca94-d081-4385-82f8-dc1db4073612,6e698671-85b9-4fcb-979d-1293e1660a90,0,default_not_whitened,franklab_default,default
36ae9a61-8f94-4047-86b2-2aae9ece2453,f294456f-c177-42b4-9895-dbcb98757337,0,default_not_whitened,franklab_default,default


Entries for IM-1478_20220726_.nwb in sgs.SpikeSortingOutput


[15:13:44][WARNING] Spyglass: V0 requires artifact restrict. Ignoring restrict_by_artifact


merge_id,source
01575f8f-ca3f-a7c6-e68c-2d24ed3666b3,CurationV1
06f09f86-93b3-1f74-bc32-31edf2e3b863,CurationV1
09360af7-7244-7cd4-466c-ff290366318c,CurationV1
0e99a76b-31bf-aa50-55cf-55193998dad6,CurationV1
1078ff7b-4a11-b3d6-b81f-12b41d41d52b,CurationV1
12177cd9-e60d-66f3-d629-e5d249a06401,CurationV1
1b144492-8c92-d382-9e22-c812ae92ba15,CurationV1
1bc939bc-9189-f91a-1337-528b46d6356b,CurationV1
2f8f8149-cd9d-56a8-3bd4-54662980c0c5,CurationV1
310e8636-cc4c-d368-935a-c7487de886e5,CurationV1


## Define sort groups and extract recordings

For now we generally sort electrodes on the same shank together. 


I have also written some custom functions that allow us to set SortGroups based on different attributes, so we may have way more `SortGroups` than shanks (if we started by one `SortGroup` per shank, then chose custom ones with different numbers). Generally when we define custom `SortGroups` we'll start from higher numbers to make it clear that they are separate from the ones auto-assigned by `sgs.SortGroup.set_group_by_shank` (e.g. if `set_group_by_shank` assigns groups 0-24 for 25 good shanks, we might choose to start our custom `SortGroups` at id 40 to leave a clear gap in between them)


If you do this (have multiple sets of `SortGroups`), remember to choose the actual sort groups you want and don't just use all of them.

In [4]:
existing_sort_group_ids = (sgs.SortGroup & {"nwb_file_name": nwb_file_name}).fetch(
    "sort_group_id"
)
print(f"All existing sort group ids for this nwb: {existing_sort_group_ids}")

# Quick check before we overwrite everything!
# If no SortGroups exist yet, we generally start with setting them by shank.
if existing_sort_group_ids.size == 0:
    sgs.SortGroup.set_group_by_shank(nwb_file_name=nwb_file_name)

All existing sort group ids for this nwb: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61]


Choose the sort group ids we actually want to use!

In [5]:
# # Use all of them:
# sort_group_ids = (sgs.SortGroup & {"nwb_file_name": nwb_file_name}).fetch("sort_group_id")

# Or pick a few:
# NOTE: For IM-1478_20220726_.nwb I'm using SortGroups 1-25 (output by set_group_by_shank)
# Yang-Sun uses custom groups with ids 40+ for clusterless decoding
sort_group_ids = list(range(25))

print(f"Using sort_group_ids: {sort_group_ids}")

Using sort_group_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


## Preprocessing

Filter and reference the recording so that we isolate the spike band data.

In [6]:
# Define and insert a key for each sort group / interval / parameters you want to sort

group_keys = []
for sort_group_id in sort_group_ids:
    key = {
        "nwb_file_name": nwb_file_name,
        "sort_group_id": sort_group_id,
        "interval_list_name": "00_r1",
        "preproc_param_name": preproc_param_name,
        "team_name": "Berke lab and friends",
    }
    # Insert into the selection table
    sgs.SpikeSortingRecordingSelection.insert_selection(key)

    # Grab the primary key (recording_id) and add to our list so we can insert into SpikeSortingRecording
    group_keys.append((sgs.SpikeSortingRecordingSelection & key).fetch1("KEY"))

# Look at everything we inserted!
display(sgs.SpikeSortingRecordingSelection & group_keys)

print("Group keys:")
print(group_keys)

[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already in

recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name
0867803c-2e78-4ac6-b4e2-6b085ca255ed,IM-1478_20220726_.nwb,4,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
0d2e1d88-a0a1-4b08-8108-1003a19914e9,IM-1478_20220726_.nwb,7,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
26067b42-3b2e-49c1-98f8-57a1bb6b23a4,IM-1478_20220726_.nwb,9,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
29ad71d5-4fbc-4a4e-8931-9dffe9190a9c,IM-1478_20220726_.nwb,6,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
34ab2436-3685-4fcf-aef5-1b795b00f237,IM-1478_20220726_.nwb,0,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
44d30a68-7fe8-4428-94dc-72232ab66ffe,IM-1478_20220726_.nwb,10,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
4b441296-09cb-46f4-893e-269aca33df1b,IM-1478_20220726_.nwb,15,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
51282a32-32ed-4792-ae0c-88f3418bdf55,IM-1478_20220726_.nwb,5,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
5227a486-5c5a-451b-bd3d-3984fa6d822c,IM-1478_20220726_.nwb,8,00_r1,franklab_tetrode_hippocampus,Berke lab and friends
558a99a8-321e-49c2-8eaa-2fb7d04c758a,IM-1478_20220726_.nwb,24,00_r1,franklab_tetrode_hippocampus,Berke lab and friends


Group keys:
[{'recording_id': UUID('34ab2436-3685-4fcf-aef5-1b795b00f237')}, {'recording_id': UUID('8ecaf134-7d00-40c4-b2a0-92be69d2a187')}, {'recording_id': UUID('a755fa00-bb57-4a00-8885-b194f2a40600')}, {'recording_id': UUID('d58b7cdc-da75-4547-ade1-171c66739dff')}, {'recording_id': UUID('0867803c-2e78-4ac6-b4e2-6b085ca255ed')}, {'recording_id': UUID('51282a32-32ed-4792-ae0c-88f3418bdf55')}, {'recording_id': UUID('29ad71d5-4fbc-4a4e-8931-9dffe9190a9c')}, {'recording_id': UUID('0d2e1d88-a0a1-4b08-8108-1003a19914e9')}, {'recording_id': UUID('5227a486-5c5a-451b-bd3d-3984fa6d822c')}, {'recording_id': UUID('26067b42-3b2e-49c1-98f8-57a1bb6b23a4')}, {'recording_id': UUID('44d30a68-7fe8-4428-94dc-72232ab66ffe')}, {'recording_id': UUID('b4691cb3-de54-4a8c-8ed3-fe585320a4f5')}, {'recording_id': UUID('57b8bebb-ecb9-4ad3-bc90-16580ffe3d62')}, {'recording_id': UUID('df8d64c4-5241-4021-b0f5-1185d570d2d3')}, {'recording_id': UUID('9e47ae05-cba0-4e77-baa7-27a516d4add1')}, {'recording_id': UUID('4b44

Now call the `populate` method of `SpikeSortingRecording`. 

Instead of just calling `sgs.SpikeSortingRecording.populate(group_keys)` with all group_keys, we only populate the missing keys.

In [7]:
# Populate SpikeSortingRecording for all group_keys (ignoring ones already populated)

# Print the set of all possible group keys
print(f"There are {len(group_keys)} keys: {group_keys}")

# Get the set of already-populated keys in SpikeSortingRecording
existing_keys = (sgs.SpikeSortingRecording & group_keys).fetch('KEY', as_dict=True)
print(f"There are {len(existing_keys)} already in in SpikeSortingRecording: {existing_keys}")

# Find missing keys
missing_keys = [key for key in group_keys if key not in existing_keys]
print(f"There are {len(missing_keys)} missing keys: {missing_keys}")

# Populate only missing entries
if missing_keys:
    sgs.SpikeSortingRecording().populate(missing_keys)
else:
    print("All group keys already populated.")

There are 25 keys: [{'recording_id': UUID('34ab2436-3685-4fcf-aef5-1b795b00f237')}, {'recording_id': UUID('8ecaf134-7d00-40c4-b2a0-92be69d2a187')}, {'recording_id': UUID('a755fa00-bb57-4a00-8885-b194f2a40600')}, {'recording_id': UUID('d58b7cdc-da75-4547-ade1-171c66739dff')}, {'recording_id': UUID('0867803c-2e78-4ac6-b4e2-6b085ca255ed')}, {'recording_id': UUID('51282a32-32ed-4792-ae0c-88f3418bdf55')}, {'recording_id': UUID('29ad71d5-4fbc-4a4e-8931-9dffe9190a9c')}, {'recording_id': UUID('0d2e1d88-a0a1-4b08-8108-1003a19914e9')}, {'recording_id': UUID('5227a486-5c5a-451b-bd3d-3984fa6d822c')}, {'recording_id': UUID('26067b42-3b2e-49c1-98f8-57a1bb6b23a4')}, {'recording_id': UUID('44d30a68-7fe8-4428-94dc-72232ab66ffe')}, {'recording_id': UUID('b4691cb3-de54-4a8c-8ed3-fe585320a4f5')}, {'recording_id': UUID('57b8bebb-ecb9-4ad3-bc90-16580ffe3d62')}, {'recording_id': UUID('df8d64c4-5241-4021-b0f5-1185d570d2d3')}, {'recording_id': UUID('9e47ae05-cba0-4e77-baa7-27a516d4add1')}, {'recording_id': UUI

In [8]:
# Make sure everything worked!
sgs.SpikeSortingRecording() & group_keys

recording_id,analysis_file_name name of the file,object_id Object ID for the processed recording in NWB file,electrodes_id Object ID for the processed electrodes,hash Hash of the NWB file
0867803c-2e78-4ac6-b4e2-6b085ca255ed,IM-1478_20220726_7JRFV48Z1Z.nwb,7631af76-aee5-43d2-bb17-6ad0de9c3589,1216005c-303d-413c-b83c-048becc338f3,3dba08fb6fc6720ac5fd168411b72eb0
0d2e1d88-a0a1-4b08-8108-1003a19914e9,IM-1478_20220726_0TRJO6BVLG.nwb,4992e6c3-dce4-477a-8bc0-51d2983f226e,ee32fb44-9374-409b-8a32-d0b9d44fc075,d60ed0f27b170309ce11c1996636d726
26067b42-3b2e-49c1-98f8-57a1bb6b23a4,IM-1478_20220726_075WTHHEY7.nwb,24bb4873-88d3-4e82-be3b-5d68aed3c617,0a577a71-20ae-4cc3-9a48-663d3731ca10,bcf72ca13bf2037d5574a140c234f546
29ad71d5-4fbc-4a4e-8931-9dffe9190a9c,IM-1478_20220726_1U9DIBEQCC.nwb,16658107-052d-4f47-baa6-dd17110fe471,ed9deb67-3b29-4f7a-be5f-f741e52e7634,207a3e445669c5811e2d021e5645d855
34ab2436-3685-4fcf-aef5-1b795b00f237,IM-1478_20220726_QEEZ8GSEMP.nwb,56e4ea9b-c2b8-43bd-b4ed-be42f63a72e2,909a1c66-264a-40c4-8511-a867f8e2f41a,4126b8d0a686a5f713fd65368ca8b293
44d30a68-7fe8-4428-94dc-72232ab66ffe,IM-1478_20220726_0MJFRVSV4M.nwb,15b2be77-d5cb-4d5f-9274-628c8a5ea17a,69ae7807-80a9-4e2f-be76-ad331c02fb28,de5d5540c53c8667069d2fd99acf38b7
4b441296-09cb-46f4-893e-269aca33df1b,IM-1478_20220726_6AY9YQ6RFD.nwb,3623ad66-5233-46c1-83a3-82499abc6653,464f32c0-3f54-4f7c-a1ed-3d934c082991,43c69a0c3b00db66d2d39de661de86f2
51282a32-32ed-4792-ae0c-88f3418bdf55,IM-1478_20220726_0LTMS9IPXG.nwb,e5661178-2445-491a-883d-32da6eeeafc9,e2d88e4e-4e76-4c59-a854-53e5587bc29d,215206d201b0786b1f4c2e8dcb5134e5
5227a486-5c5a-451b-bd3d-3984fa6d822c,IM-1478_20220726_F7EMMDP93N.nwb,83bac936-9f6b-4b21-8770-657ea3798db7,854a62b8-8a71-4549-bb4a-958a837389f3,7ab03f6dab09d89cb42864fda85c9a64
558a99a8-321e-49c2-8eaa-2fb7d04c758a,IM-1478_20220726_YHMP5AE2HI.nwb,ecf2747a-42d3-466f-aa6d-13a994d70e14,3dcd4864-f9a2-400b-831e-31b4681ed5e3,09aa2a5dffd0c4f9bee89ad266654a50


## Artifact Detection

In [9]:
print(group_keys)

artifact_detection_keys = []

for group_key in group_keys:
    key = {
        "recording_id": group_key["recording_id"],
        "artifact_param_name": artifact_param_name,
    }
    # Insert into the selection table
    sgs.ArtifactDetectionSelection.insert_selection(key)

    # Grab the primary key (artifact_id) and add to our list so we can insert into ArtifactDetection
    artifact_detection_keys.append((sgs.ArtifactDetectionSelection & key).fetch1("KEY"))

# Look at everything we inserted!
display(sgs.ArtifactDetectionSelection() & artifact_detection_keys)

print("Artifact detection keys:")
print(artifact_detection_keys)

[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.


[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.


[{'recording_id': UUID('34ab2436-3685-4fcf-aef5-1b795b00f237')}, {'recording_id': UUID('8ecaf134-7d00-40c4-b2a0-92be69d2a187')}, {'recording_id': UUID('a755fa00-bb57-4a00-8885-b194f2a40600')}, {'recording_id': UUID('d58b7cdc-da75-4547-ade1-171c66739dff')}, {'recording_id': UUID('0867803c-2e78-4ac6-b4e2-6b085ca255ed')}, {'recording_id': UUID('51282a32-32ed-4792-ae0c-88f3418bdf55')}, {'recording_id': UUID('29ad71d5-4fbc-4a4e-8931-9dffe9190a9c')}, {'recording_id': UUID('0d2e1d88-a0a1-4b08-8108-1003a19914e9')}, {'recording_id': UUID('5227a486-5c5a-451b-bd3d-3984fa6d822c')}, {'recording_id': UUID('26067b42-3b2e-49c1-98f8-57a1bb6b23a4')}, {'recording_id': UUID('44d30a68-7fe8-4428-94dc-72232ab66ffe')}, {'recording_id': UUID('b4691cb3-de54-4a8c-8ed3-fe585320a4f5')}, {'recording_id': UUID('57b8bebb-ecb9-4ad3-bc90-16580ffe3d62')}, {'recording_id': UUID('df8d64c4-5241-4021-b0f5-1185d570d2d3')}, {'recording_id': UUID('9e47ae05-cba0-4e77-baa7-27a516d4add1')}, {'recording_id': UUID('4b441296-09cb-46

[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:45][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already inserted.
[15:13:46][WARNING] Spyglass: Similar row(s) already in

artifact_id,recording_id,artifact_param_name
0e848e3d-74d1-4228-bc58-23693b42a787,7d5ff076-1bbe-4118-8e92-26479f61cb69,ampl_1000_z_30_prop_075_1ms
1cb9426e-66a1-49c0-83d5-62ac8fdc9f3c,0d2e1d88-a0a1-4b08-8108-1003a19914e9,ampl_1000_z_30_prop_075_1ms
39bff494-ab70-47eb-9b79-abdf55127f42,a755fa00-bb57-4a00-8885-b194f2a40600,ampl_1000_z_30_prop_075_1ms
3ee24ae0-f601-45d9-9374-0972dd713d11,dd2afb0c-83e3-4f57-9b0d-75d73d69f780,ampl_1000_z_30_prop_075_1ms
4ec908ac-f242-42d5-9fc8-5b6063028ed1,51282a32-32ed-4792-ae0c-88f3418bdf55,ampl_1000_z_30_prop_075_1ms
50016ae9-ed50-436f-89d4-c1a4a97e9927,44d30a68-7fe8-4428-94dc-72232ab66ffe,ampl_1000_z_30_prop_075_1ms
58ce55ea-6471-49ce-a8e4-8e4bbcc2ce77,0867803c-2e78-4ac6-b4e2-6b085ca255ed,ampl_1000_z_30_prop_075_1ms
5af72af4-9ec5-4847-881a-579dd4a89b5d,e60131dd-e3a3-44cc-b36e-a3761765d428,ampl_1000_z_30_prop_075_1ms
6825da93-dc9e-47a4-ad2b-06c685085885,df8d64c4-5241-4021-b0f5-1185d570d2d3,ampl_1000_z_30_prop_075_1ms
698d451d-2b6f-4ad7-a413-d7132e35fe22,b4691cb3-de54-4a8c-8ed3-fe585320a4f5,ampl_1000_z_30_prop_075_1ms


Artifact detection keys:
[{'artifact_id': UUID('a985bb6a-d502-4545-ba6a-6099291b1b8e')}, {'artifact_id': UUID('978a937a-e688-4c15-8398-b6ce8c36a61a')}, {'artifact_id': UUID('39bff494-ab70-47eb-9b79-abdf55127f42')}, {'artifact_id': UUID('e313b290-a965-4f6b-8081-990a8ad81a6a')}, {'artifact_id': UUID('58ce55ea-6471-49ce-a8e4-8e4bbcc2ce77')}, {'artifact_id': UUID('4ec908ac-f242-42d5-9fc8-5b6063028ed1')}, {'artifact_id': UUID('e20e53aa-8b22-417e-a575-302898172607')}, {'artifact_id': UUID('1cb9426e-66a1-49c0-83d5-62ac8fdc9f3c')}, {'artifact_id': UUID('a30fcdc7-2301-488f-92f0-cdb8f7c3aefc')}, {'artifact_id': UUID('f68ac3c4-75ac-4df3-a0e9-d1395b723a52')}, {'artifact_id': UUID('50016ae9-ed50-436f-89d4-c1a4a97e9927')}, {'artifact_id': UUID('698d451d-2b6f-4ad7-a413-d7132e35fe22')}, {'artifact_id': UUID('d20e777a-e5d7-4692-92c3-5fefe59d767e')}, {'artifact_id': UUID('6825da93-dc9e-47a4-ad2b-06c685085885')}, {'artifact_id': UUID('c905af86-a6f6-478e-a36d-68f3cd51d673')}, {'artifact_id': UUID('c6b986f

Now call the `populate` method of `ArtifactDetection`.

Instead of just calling `sgs.ArtifactDetection.populate(artifact_detection_keys)` with all artifact_detection_keys, we only populate the missing keys.

In [10]:
# Populate ArtifactDetection for all artifact_detection_keys (ignoring ones already populated)

# Print the set of all possible keys
print(f"There are {len(artifact_detection_keys)} keys: {artifact_detection_keys}")

# Get the set of already-populated keys in ArtifactDetection
existing_keys = (sgs.ArtifactDetection & artifact_detection_keys).fetch('KEY', as_dict=True)
print(f"There are {len(existing_keys)} already in in ArtifactDetection: {existing_keys}")

# Find missing keys
missing_keys = [key for key in artifact_detection_keys if key not in existing_keys]
print(f"There are {len(missing_keys)} missing keys: {missing_keys}")

# Populate only missing entries
if missing_keys:
    sgs.ArtifactDetection().populate(missing_keys)
else:
    print("All group keys already populated.")

There are 25 keys: [{'artifact_id': UUID('a985bb6a-d502-4545-ba6a-6099291b1b8e')}, {'artifact_id': UUID('978a937a-e688-4c15-8398-b6ce8c36a61a')}, {'artifact_id': UUID('39bff494-ab70-47eb-9b79-abdf55127f42')}, {'artifact_id': UUID('e313b290-a965-4f6b-8081-990a8ad81a6a')}, {'artifact_id': UUID('58ce55ea-6471-49ce-a8e4-8e4bbcc2ce77')}, {'artifact_id': UUID('4ec908ac-f242-42d5-9fc8-5b6063028ed1')}, {'artifact_id': UUID('e20e53aa-8b22-417e-a575-302898172607')}, {'artifact_id': UUID('1cb9426e-66a1-49c0-83d5-62ac8fdc9f3c')}, {'artifact_id': UUID('a30fcdc7-2301-488f-92f0-cdb8f7c3aefc')}, {'artifact_id': UUID('f68ac3c4-75ac-4df3-a0e9-d1395b723a52')}, {'artifact_id': UUID('50016ae9-ed50-436f-89d4-c1a4a97e9927')}, {'artifact_id': UUID('698d451d-2b6f-4ad7-a413-d7132e35fe22')}, {'artifact_id': UUID('d20e777a-e5d7-4692-92c3-5fefe59d767e')}, {'artifact_id': UUID('6825da93-dc9e-47a4-ad2b-06c685085885')}, {'artifact_id': UUID('c905af86-a6f6-478e-a36d-68f3cd51d673')}, {'artifact_id': UUID('c6b986ff-6c6a

In [11]:
# Make sure everything worked!
display(sgs.ArtifactDetection() & artifact_detection_keys)

artifact_id
0e848e3d-74d1-4228-bc58-23693b42a787
1cb9426e-66a1-49c0-83d5-62ac8fdc9f3c
39bff494-ab70-47eb-9b79-abdf55127f42
3ee24ae0-f601-45d9-9374-0972dd713d11
4ec908ac-f242-42d5-9fc8-5b6063028ed1
50016ae9-ed50-436f-89d4-c1a4a97e9927
58ce55ea-6471-49ce-a8e4-8e4bbcc2ce77
5af72af4-9ec5-4847-881a-579dd4a89b5d
6825da93-dc9e-47a4-ad2b-06c685085885
698d451d-2b6f-4ad7-a413-d7132e35fe22


## Run Spike Sorting

The spike sorting pipeline is powered by `spikeinterface`, a community-developed Python package that enables one to easily apply multiple spike sorters to a single recording. Some spike sorters have special requirements, such as GPU. Others need to be installed separately from spyglass. In the Frank lab, we have been using `mountainsort4`, though the pipeline have been tested with `mountainsort5`, `kilosort2_5`, `kilosort3`, and `ironclust` as well.

When using `mountainsort5`, make sure to run `pip install mountainsort5`. `kilosort2_5`, `kilosort3`, and `ironclust` are MATLAB-based, but we can run these without having to install MATLAB thanks to `spikeinterface`. It does require downloading additional files (as singularity containers) so make sure to do `pip install spython`. These sorters also require GPU access, so also do ` pip install cuda-python` (and make sure your computer does have a GPU). 

In [12]:
# Insert into SpikeSortingSelection

spike_sorting_keys = []

for group_key in group_keys:
    # Sometimes not all of these correctly populated in ArtifactDetectionSelection but we want to move forward anyway.
    # So we do a check that the interval list actually exists so we can move forward with the ones that did
    art_id = (sgs.ArtifactDetectionSelection & {"recording_id": group_key["recording_id"]}).fetch1("artifact_id")
    interval_list_entry = (sgc.IntervalList() & {"interval_list_name": str(art_id)})
    if len(interval_list_entry.fetch()) == 0:
        print(f"No interval list entry for {art_id}, skipping.")
        continue

    ss_key = {
        "recording_id": group_key["recording_id"],
        "sorter": sorter,
        "nwb_file_name": nwb_file_name,
        "interval_list_name": str(art_id),
        "sorter_param_name": sorter_param_name,
    }
    # Insert into the selection table
    sgs.SpikeSortingSelection.insert_selection(ss_key)

    # Grab the primary key (sorting_id) and add to our list so we can insert into SpikeSorting
    spike_sorting_keys.append((sgs.SpikeSortingSelection & ss_key).proj().fetch1("KEY"))

# Look at everything we inserted!
display(sgs.SpikeSortingSelection() & spike_sorting_keys)

print("Spike sorting keys:")
print(spike_sorting_keys)

[15:13:48][INFO] Spyglass: Similar row(s) already inserted.
[15:13:49][INFO] Spyglass: Similar row(s) already inserted.
[15:13:49][INFO] Spyglass: Similar row(s) already inserted.
[15:13:50][INFO] Spyglass: Similar row(s) already inserted.
[15:13:50][INFO] Spyglass: Similar row(s) already inserted.
[15:13:50][INFO] Spyglass: Similar row(s) already inserted.
[15:13:51][INFO] Spyglass: Similar row(s) already inserted.
[15:13:51][INFO] Spyglass: Similar row(s) already inserted.
[15:13:52][INFO] Spyglass: Similar row(s) already inserted.
[15:13:52][INFO] Spyglass: Similar row(s) already inserted.
[15:13:52][INFO] Spyglass: Similar row(s) already inserted.
[15:13:53][INFO] Spyglass: Similar row(s) already inserted.
[15:13:53][INFO] Spyglass: Similar row(s) already inserted.
[15:13:53][INFO] Spyglass: Similar row(s) already inserted.
[15:13:54][INFO] Spyglass: Similar row(s) already inserted.
[15:13:54][INFO] Spyglass: Similar row(s) already inserted.
[15:13:54][INFO] Spyglass: Similar row(s

sorting_id,recording_id,sorter,sorter_param_name,nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list
182eb872-395f-4248-97be-d481f08c9154,dd2afb0c-83e3-4f57-9b0d-75d73d69f780,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,3ee24ae0-f601-45d9-9374-0972dd713d11
20fa485c-bdbb-456a-9e83-17a9bea8b2df,e2d0a59a-713e-4b24-a22e-5d8e63ae3463,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,e5eb99f0-0574-4c60-9933-e4e0869972b1
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,847eb270-89e4-472b-bd8d-15438dd8ffd4,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,9e90129a-7d22-4bd9-8023-ccfc2bdbb45b
38635c94-2bea-493e-998d-4ffd420f070b,b119e22f-7b24-43e9-a5da-d594f1e5848d,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,faa3d190-4b5b-449c-89f4-331d8fcdacd6
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,29ad71d5-4fbc-4a4e-8931-9dffe9190a9c,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,e20e53aa-8b22-417e-a575-302898172607
4a006737-9754-41de-8bf2-fd46caa68ec1,44d30a68-7fe8-4428-94dc-72232ab66ffe,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,50016ae9-ed50-436f-89d4-c1a4a97e9927
4ca04e9c-3352-48e0-8beb-a07281c72a1c,9e47ae05-cba0-4e77-baa7-27a516d4add1,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,c905af86-a6f6-478e-a36d-68f3cd51d673
59ce8c71-5946-4ad6-9824-a78a5492e5b4,e60131dd-e3a3-44cc-b36e-a3761765d428,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,5af72af4-9ec5-4847-881a-579dd4a89b5d
6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf,26067b42-3b2e-49c1-98f8-57a1bb6b23a4,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,f68ac3c4-75ac-4df3-a0e9-d1395b723a52
6e698671-85b9-4fcb-979d-1293e1660a90,34ab2436-3685-4fcf-aef5-1b795b00f237,mountainsort4,franklab_tetrode_hippocampus_30KHz,IM-1478_20220726_.nwb,a985bb6a-d502-4545-ba6a-6099291b1b8e


Spike sorting keys:
[{'sorting_id': UUID('6e698671-85b9-4fcb-979d-1293e1660a90')}, {'sorting_id': UUID('e2417f7f-b7f4-4698-949f-03582177a710')}, {'sorting_id': UUID('cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4')}, {'sorting_id': UUID('f294456f-c177-42b4-9895-dbcb98757337')}, {'sorting_id': UUID('b7c8c657-05ef-4b15-9a0f-a00218d5163c')}, {'sorting_id': UUID('7617faba-e28c-4846-b04e-3dbdac4f4540')}, {'sorting_id': UUID('40f8ce6b-c9c2-4af4-acaf-da32a4f73b51')}, {'sorting_id': UUID('c9da7594-c6ee-43b4-9a1e-7c9bc21906f4')}, {'sorting_id': UUID('a28382e1-eb6d-4411-95e3-c8c7a6fe2798')}, {'sorting_id': UUID('6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf')}, {'sorting_id': UUID('4a006737-9754-41de-8bf2-fd46caa68ec1')}, {'sorting_id': UUID('bd374141-ba9f-48d0-a83d-282649e94b17')}, {'sorting_id': UUID('81076d23-cdf1-4881-a726-a3d2d44090f6')}, {'sorting_id': UUID('aa35a78e-3f18-4e47-bf62-4bd47ed2f79f')}, {'sorting_id': UUID('4ca04e9c-3352-48e0-8beb-a07281c72a1c')}, {'sorting_id': UUID('b563f306-5d29-4279-8d6e-c97c

Now call the `populate` method of `SpikeSorting`.

Instead of just calling `sgs.SpikeSorting.populate(spike_sorting_keys)` with all spike_sorting_keys, we only populate the missing keys.

In [13]:
# Populate SpikeSorting for all spike_sorting_keys (ignoring ones already populated)

# Print the set of all possible keys
print(f"There are {len(spike_sorting_keys)} keys: {spike_sorting_keys}")

# Get the set of already-populated keys in sgs.SpikeSorting
existing_keys = (sgs.SpikeSorting & spike_sorting_keys).fetch('KEY', as_dict=True)
print(f"There are {len(existing_keys)} already in in sgs.SpikeSorting: {existing_keys}")

# Find missing keys
missing_keys = [key for key in spike_sorting_keys if key not in existing_keys]
print(f"There are {len(missing_keys)} missing keys: {missing_keys}")

# Populate only missing entries
if missing_keys:
    sgs.SpikeSorting.populate(missing_keys)
else:
    print("All group keys already populated.")

There are 25 keys: [{'sorting_id': UUID('6e698671-85b9-4fcb-979d-1293e1660a90')}, {'sorting_id': UUID('e2417f7f-b7f4-4698-949f-03582177a710')}, {'sorting_id': UUID('cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4')}, {'sorting_id': UUID('f294456f-c177-42b4-9895-dbcb98757337')}, {'sorting_id': UUID('b7c8c657-05ef-4b15-9a0f-a00218d5163c')}, {'sorting_id': UUID('7617faba-e28c-4846-b04e-3dbdac4f4540')}, {'sorting_id': UUID('40f8ce6b-c9c2-4af4-acaf-da32a4f73b51')}, {'sorting_id': UUID('c9da7594-c6ee-43b4-9a1e-7c9bc21906f4')}, {'sorting_id': UUID('a28382e1-eb6d-4411-95e3-c8c7a6fe2798')}, {'sorting_id': UUID('6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf')}, {'sorting_id': UUID('4a006737-9754-41de-8bf2-fd46caa68ec1')}, {'sorting_id': UUID('bd374141-ba9f-48d0-a83d-282649e94b17')}, {'sorting_id': UUID('81076d23-cdf1-4881-a726-a3d2d44090f6')}, {'sorting_id': UUID('aa35a78e-3f18-4e47-bf62-4bd47ed2f79f')}, {'sorting_id': UUID('4ca04e9c-3352-48e0-8beb-a07281c72a1c')}, {'sorting_id': UUID('b563f306-5d29-4279-8d6e-c97cb

In [14]:
# Make sure everything worked!
display(sgs.SpikeSorting() & spike_sorting_keys)

sorting_id,analysis_file_name name of the file,object_id Object ID for the sorting in NWB file,"time_of_sort in Unix time, to the nearest second"
182eb872-395f-4248-97be-d481f08c9154,IM-1478_20220726_XTVSKAFMZD.nwb,9b48fe55-a9a2-49af-bf67-b6ac6d84e400,1762368782
20fa485c-bdbb-456a-9e83-17a9bea8b2df,IM-1478_20220726_6GXAOZUEEA.nwb,159c6b8c-fff6-4ee1-b13c-160491afceff,1762369236
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,IM-1478_20220726_XSRR71LU4H.nwb,455b158d-59ed-40f9-8f0d-7efa5b864e5a,1762369671
38635c94-2bea-493e-998d-4ffd420f070b,IM-1478_20220726_KMV7N6Z25G.nwb,065230b0-defc-47f8-8481-0ea991aca6e1,1762370040
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,IM-1478_20220726_ZVMYPEUHKJ.nwb,a81c6b21-d8f1-4141-8593-43e922cf1249,1762370462
4a006737-9754-41de-8bf2-fd46caa68ec1,IM-1478_20220726_ETLZ47VD96.nwb,dcc4a4ef-4db0-43ee-b3aa-be033c0c2ece,1762370915
4ca04e9c-3352-48e0-8beb-a07281c72a1c,IM-1478_20220726_3ZEGX135Q7.nwb,350ff84e-0205-4801-bb7d-230bf1ca09cf,1762371322
59ce8c71-5946-4ad6-9824-a78a5492e5b4,IM-1478_20220726_3TLAXQGM6G.nwb,b07894ab-3538-4f8d-9b48-e781303043ff,1762371726
6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf,IM-1478_20220726_99RGFN6JPX.nwb,69d9cb29-a4d6-4cc9-8b23-021e7d575958,1762372064
6e698671-85b9-4fcb-979d-1293e1660a90,IM-1478_20220726_9PML42GLTN.nwb,1d28944d-5ab6-4ab9-a214-2b4ba630c393,1762372377


The spike sorting results (spike times of detected units) are saved in an NWB file. We can access this in two ways. First, we can access it via the `fetch_nwb` method, which allows us to directly access the spike times saved in the `units` table of the NWB file. Second, we can access it as a `spikeinterface.NWBSorting` object. This allows us to take advantage of the rich APIs of `spikeinterface` to further analyze the sorting. 

In [15]:
# for ss_key in spike_sorting_keys:
#     sorting_nwb = (sgs.SpikeSorting & ss_key).fetch_nwb()
#     sorting_si = sgs.SpikeSorting.get_sorting(ss_key)

Note that the spike times of `fetch_nwb` is in units of seconds aligned with the timestamps of the recording. The spike times of the `spikeinterface.NWBSorting` object is in units of samples (as is generally true for sorting objects in `spikeinterface`).

## Automatic Curation

Next step is to curate the results of spike sorting. This is often necessary because spike sorting algorithms are not perfect;
they often return clusters that are clearly not biological in origin, and sometimes oversplit clusters that should have been merged.
We have two main ways of curating spike sorting: by computing quality metrics followed by thresholding, and manually applying curation labels.
To do either, we first insert the spike sorting to `CurationV1` using `insert_curation` method.


In [16]:
curation_key_list = []

for ss_key in spike_sorting_keys:

    # Check if this sorting_id has already been inserted with curation_id=1
    initial_curation_key = {"sorting_id": str(ss_key["sorting_id"]), "curation_id": 0}
    initial_curation_entry = (sgs.CurationV1() & initial_curation_key)

    # If it hasn't been inserted yet, insert into the curation table
    if len(initial_curation_entry.fetch()) == 0:
        sgs.CurationV1.insert_curation(
            sorting_id=str(ss_key["sorting_id"]),
            description="initial automatic curation",
        )
    else:
        print(f"Entry for {initial_curation_key} already exists in sgs.CurationV1")

    curation_key_list.append(initial_curation_key)

# Look at everything we inserted!
display(sgs.CurationV1() & curation_key_list)

print("Curation keys (initial automatic curation):")
print(curation_key_list)

Entry for {'sorting_id': '6e698671-85b9-4fcb-979d-1293e1660a90', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'e2417f7f-b7f4-4698-949f-03582177a710', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'f294456f-c177-42b4-9895-dbcb98757337', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'b7c8c657-05ef-4b15-9a0f-a00218d5163c', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': '7617faba-e28c-4846-b04e-3dbdac4f4540', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': '40f8ce6b-c9c2-4af4-acaf-da32a4f73b51', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'c9da7594-c6ee-43b4-9a1e-7c9bc21906f4', 'curation_id': 0} already exists in sgs.CurationV1
Entry for {'sorting_id': 'a28382e1-eb6d-4411-95e3-c8c7a6fe2798', 'curati

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
182eb872-395f-4248-97be-d481f08c9154,0,-1,IM-1478_20220726_CAC88V34XP.nwb,36bbc865-b980-41b1-8003-23e3b3e25379,0,initial automatic curation
20fa485c-bdbb-456a-9e83-17a9bea8b2df,0,-1,IM-1478_20220726_2LQZTIRXZT.nwb,aadc3255-3b43-4dfe-afbf-7409479f90ad,0,initial automatic curation
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,0,-1,IM-1478_20220726_CX7QVUQ6AQ.nwb,1388a9b6-a219-450f-a9fc-99623551683f,0,initial automatic curation
38635c94-2bea-493e-998d-4ffd420f070b,0,-1,IM-1478_20220726_EFAK43PR86.nwb,77be9e52-3d8d-47e4-a61d-1859b37fbfe9,0,initial automatic curation
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,0,-1,IM-1478_20220726_URF84UP1Z3.nwb,3e33ecce-864d-4e11-b80e-1cbfdf16a5dd,0,initial automatic curation
4a006737-9754-41de-8bf2-fd46caa68ec1,0,-1,IM-1478_20220726_BXR60B8RGD.nwb,a782861e-2518-4b72-af21-11e980669e1b,0,initial automatic curation
4ca04e9c-3352-48e0-8beb-a07281c72a1c,0,-1,IM-1478_20220726_ZOV7ONADXC.nwb,75872930-0ddd-4079-9e29-4f40d9945041,0,initial automatic curation
59ce8c71-5946-4ad6-9824-a78a5492e5b4,0,-1,IM-1478_20220726_L3D59AGCKD.nwb,bd1e9737-5fe7-4adf-8570-384d2f98c580,0,initial automatic curation
6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf,0,-1,IM-1478_20220726_5SL3NNB18D.nwb,8391a8ba-30f9-45c3-a712-0e215f6bd169,0,initial automatic curation
6e698671-85b9-4fcb-979d-1293e1660a90,0,-1,IM-1478_20220726_8Q2W3IUCY1.nwb,ac0ad6da-c80e-4bbb-b635-12a8e8252f75,0,initial automatic curation


Curation keys (initial automatic curation):
[{'sorting_id': '6e698671-85b9-4fcb-979d-1293e1660a90', 'curation_id': 0}, {'sorting_id': 'e2417f7f-b7f4-4698-949f-03582177a710', 'curation_id': 0}, {'sorting_id': 'cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4', 'curation_id': 0}, {'sorting_id': 'f294456f-c177-42b4-9895-dbcb98757337', 'curation_id': 0}, {'sorting_id': 'b7c8c657-05ef-4b15-9a0f-a00218d5163c', 'curation_id': 0}, {'sorting_id': '7617faba-e28c-4846-b04e-3dbdac4f4540', 'curation_id': 0}, {'sorting_id': '40f8ce6b-c9c2-4af4-acaf-da32a4f73b51', 'curation_id': 0}, {'sorting_id': 'c9da7594-c6ee-43b4-9a1e-7c9bc21906f4', 'curation_id': 0}, {'sorting_id': 'a28382e1-eb6d-4411-95e3-c8c7a6fe2798', 'curation_id': 0}, {'sorting_id': '6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf', 'curation_id': 0}, {'sorting_id': '4a006737-9754-41de-8bf2-fd46caa68ec1', 'curation_id': 0}, {'sorting_id': 'bd374141-ba9f-48d0-a83d-282649e94b17', 'curation_id': 0}, {'sorting_id': '81076d23-cdf1-4881-a726-a3d2d44090f6', 'curation_id

We will first do an automatic curation based on quality metrics. Under the hood, this part again makes use of `spikeinterface`. Some of the quality metrics that we often compute are the nearest neighbor isolation and noise overlap metrics, as well as SNR and ISI violation rate. For computing some of these metrics, the waveforms must be extracted and projected onto a feature space. Thus here we set the parameters for waveform extraction as well as how to curate the units based on these metrics (e.g. if `nn_noise_overlap` is greater than 0.1, mark as `noise`).

In [17]:
metric_curation_keys = []
for ss_key in spike_sorting_keys:
    mc_key = {
        "sorting_id": str(ss_key["sorting_id"]),
        "curation_id": 0,
        "waveform_param_name": waveform_param_name,
        "metric_param_name": metric_param_name,
        "metric_curation_param_name": metric_curation_param_name,
    }

    # Insert into selection table
    sgs.MetricCurationSelection.insert_selection(mc_key)

    # Grab the primary key (metric_curation_id) and add to our list so we can insert into MetricCuration
    metric_curation_keys.append((sgs.MetricCurationSelection & mc_key).fetch1("KEY"))

# Look at everything we inserted!
display(sgs.MetricCurationSelection() & metric_curation_keys)

print("Metric curation keys:")
print(metric_curation_keys)


[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57][WARNING] Spyglass: This row has already been inserted.
[15:13:57]

metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
0bc55909-59fd-405a-8a94-b41d94a27804,373f4eb7-9411-41bf-ad85-16bbaccb3dfa,0,default_not_whitened,franklab_default,default
0eaaadfd-5e07-45cd-9d46-238bca751b54,a28382e1-eb6d-4411-95e3-c8c7a6fe2798,0,default_not_whitened,franklab_default,default
1d638c7f-65c8-4560-90b9-27d423148b52,c9da7594-c6ee-43b4-9a1e-7c9bc21906f4,0,default_not_whitened,franklab_default,default
227aa0f6-46c3-4a8c-85a6-b8245306ff70,a12af572-ee69-4dac-ae34-ec7252e2a92d,0,default_not_whitened,franklab_default,default
22c897d8-0ad8-427a-9729-48149b22e918,b7c8c657-05ef-4b15-9a0f-a00218d5163c,0,default_not_whitened,franklab_default,default
2908fb7b-ced5-4450-ac94-b86d79b8f700,b563f306-5d29-4279-8d6e-c97cbcc407e0,0,default_not_whitened,franklab_default,default
2b139787-a9f5-4bb3-8b7a-b60504a4f2f2,aa35a78e-3f18-4e47-bf62-4bd47ed2f79f,0,default_not_whitened,franklab_default,default
2ee3c57f-055b-4d79-be10-d16de8eddafe,7617faba-e28c-4846-b04e-3dbdac4f4540,0,default_not_whitened,franklab_default,default
365aca94-d081-4385-82f8-dc1db4073612,6e698671-85b9-4fcb-979d-1293e1660a90,0,default_not_whitened,franklab_default,default
36ae9a61-8f94-4047-86b2-2aae9ece2453,f294456f-c177-42b4-9895-dbcb98757337,0,default_not_whitened,franklab_default,default


Metric curation keys:
[{'metric_curation_id': UUID('365aca94-d081-4385-82f8-dc1db4073612')}, {'metric_curation_id': UUID('45fd0383-065b-4b9f-9538-b5fb455ce613')}, {'metric_curation_id': UUID('589b8185-59e2-4708-b71f-dd23a206f758')}, {'metric_curation_id': UUID('36ae9a61-8f94-4047-86b2-2aae9ece2453')}, {'metric_curation_id': UUID('22c897d8-0ad8-427a-9729-48149b22e918')}, {'metric_curation_id': UUID('2ee3c57f-055b-4d79-be10-d16de8eddafe')}, {'metric_curation_id': UUID('3c12ea06-d1df-45b5-bc2f-c442e69b9c0a')}, {'metric_curation_id': UUID('1d638c7f-65c8-4560-90b9-27d423148b52')}, {'metric_curation_id': UUID('0eaaadfd-5e07-45cd-9d46-238bca751b54')}, {'metric_curation_id': UUID('5c7d14ae-1bd4-47b0-bc32-3361111ccee6')}, {'metric_curation_id': UUID('9ba33175-8984-481a-893d-5147ba3448fd')}, {'metric_curation_id': UUID('c087d1c1-b6d9-467b-8524-b6e67ca34087')}, {'metric_curation_id': UUID('d75a1dc7-b906-42c9-933c-b1a7d5a46f9e')}, {'metric_curation_id': UUID('2b139787-a9f5-4bb3-8b7a-b60504a4f2f2')

Now call the `populate` method of `MetricCuration`.

Instead of just calling `sgs.MetricCuration.populate(metric_curation_keys)` with all metric_curation_keys, we only populate the missing keys.

In [18]:
 # Populate MetricCuration for all metric_curation_keys (ignoring ones already populated)

# Print the set of all possible keys
print(f"There are {len(metric_curation_keys)} keys: {metric_curation_keys}")

# Get the set of already-populated keys in sgs.MetricCuration
existing_keys = (sgs.MetricCuration & metric_curation_keys).fetch('KEY', as_dict=True)
print(f"There are {len(existing_keys)} already in in MetricCuration: {existing_keys}")

# Find missing keys
missing_keys = [key for key in metric_curation_keys if key not in existing_keys]
print(f"There are {len(missing_keys)} missing keys: {missing_keys}")

# Populate only missing entries
if missing_keys:
    sgs.MetricCuration().populate(missing_keys)
else:
    print("All group keys already populated.")


There are 25 keys: [{'metric_curation_id': UUID('365aca94-d081-4385-82f8-dc1db4073612')}, {'metric_curation_id': UUID('45fd0383-065b-4b9f-9538-b5fb455ce613')}, {'metric_curation_id': UUID('589b8185-59e2-4708-b71f-dd23a206f758')}, {'metric_curation_id': UUID('36ae9a61-8f94-4047-86b2-2aae9ece2453')}, {'metric_curation_id': UUID('22c897d8-0ad8-427a-9729-48149b22e918')}, {'metric_curation_id': UUID('2ee3c57f-055b-4d79-be10-d16de8eddafe')}, {'metric_curation_id': UUID('3c12ea06-d1df-45b5-bc2f-c442e69b9c0a')}, {'metric_curation_id': UUID('1d638c7f-65c8-4560-90b9-27d423148b52')}, {'metric_curation_id': UUID('0eaaadfd-5e07-45cd-9d46-238bca751b54')}, {'metric_curation_id': UUID('5c7d14ae-1bd4-47b0-bc32-3361111ccee6')}, {'metric_curation_id': UUID('9ba33175-8984-481a-893d-5147ba3448fd')}, {'metric_curation_id': UUID('c087d1c1-b6d9-467b-8524-b6e67ca34087')}, {'metric_curation_id': UUID('d75a1dc7-b906-42c9-933c-b1a7d5a46f9e')}, {'metric_curation_id': UUID('2b139787-a9f5-4bb3-8b7a-b60504a4f2f2')}, 

In [19]:
# Make sure everything worked!
display(sgs.MetricCuration() & metric_curation_keys)

metric_curation_id,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
0bc55909-59fd-405a-8a94-b41d94a27804,IM-1478_20220726_J3INJO3D50.nwb,d160613d-241f-40d0-885a-e79639385848
0eaaadfd-5e07-45cd-9d46-238bca751b54,IM-1478_20220726_EFZ23SUAOC.nwb,14498c82-f134-4688-90dc-7ba3c56623b3
1d638c7f-65c8-4560-90b9-27d423148b52,IM-1478_20220726_WQD05TBMS9.nwb,a6a24c17-6953-4aaf-86f0-d06bec6d8cf1
227aa0f6-46c3-4a8c-85a6-b8245306ff70,IM-1478_20220726_FOHELDGIYA.nwb,2f8fe8f2-bc51-40ee-9657-43bc236f45ca
22c897d8-0ad8-427a-9729-48149b22e918,IM-1478_20220726_83E3JYMB43.nwb,6fc72de9-81ff-4484-803a-1399b51a0115
2908fb7b-ced5-4450-ac94-b86d79b8f700,IM-1478_20220726_0HUIOR0K44.nwb,6caec92b-bc64-494d-ad94-cb75765c66d4
2b139787-a9f5-4bb3-8b7a-b60504a4f2f2,IM-1478_20220726_0M0JM5G4CP.nwb,d216037f-7928-45c9-9f24-a6e051d82ee8
2ee3c57f-055b-4d79-be10-d16de8eddafe,IM-1478_20220726_W20B2488RQ.nwb,fab3528f-51f0-4356-ba6d-b3952210c822
365aca94-d081-4385-82f8-dc1db4073612,IM-1478_20220726_2CFMUGNE07.nwb,6432c43e-e07b-4468-ba54-b840f74de496
36ae9a61-8f94-4047-86b2-2aae9ece2453,IM-1478_20220726_7K7EL9DSL4.nwb,b631c17a-cb22-495a-a190-bc5ce4016481


To do another round of curation, fetch the relevant info and insert back into CurationV1 using `insert_curation`.


Because this is the second round, we have `curation_id=1` and `parent_curation_id=0` (to match the `curation_id=0` of the first round we inserted)


In [20]:
curation_key_list_round2 = []

for mc_key in metric_curation_keys:

    # Check if this sorting_id has already been inserted with curation_id=1
    round_2_key = {"sorting_id": str((sgs.MetricCurationSelection & mc_key).fetch1("sorting_id")), "curation_id": 1}
    round_2_curation_entry = (sgs.CurationV1() & round_2_key)

    # If it hasn't been inserted yet, insert into the curation table for a second round
    if len(round_2_curation_entry.fetch()) == 0:
        labels = sgs.MetricCuration.get_labels(mc_key)
        merge_groups = sgs.MetricCuration.get_merge_groups(mc_key)
        metrics = sgs.MetricCuration.get_metrics(mc_key)
        sgs.CurationV1.insert_curation(
            sorting_id=(sgs.MetricCurationSelection & mc_key).fetch1("sorting_id"),
            parent_curation_id=0,
            labels=labels,
            merge_groups=merge_groups,
            metrics=metrics,
            description="after metric curation",
        )
    else:
        print(f"Entry for {round_2_key} already exists in sgs.CurationV1")

    curation_key_list_round2.append(round_2_key)

# Look at everything we inserted!
display(sgs.CurationV1() & curation_key_list_round2)

print("Curation keys (after metric curation):")
print(curation_key_list_round2)

Entry for {'sorting_id': '6e698671-85b9-4fcb-979d-1293e1660a90', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'e2417f7f-b7f4-4698-949f-03582177a710', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'f294456f-c177-42b4-9895-dbcb98757337', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'b7c8c657-05ef-4b15-9a0f-a00218d5163c', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': '7617faba-e28c-4846-b04e-3dbdac4f4540', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': '40f8ce6b-c9c2-4af4-acaf-da32a4f73b51', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'c9da7594-c6ee-43b4-9a1e-7c9bc21906f4', 'curation_id': 1} already exists in sgs.CurationV1
Entry for {'sorting_id': 'a28382e1-eb6d-4411-95e3-c8c7a6fe2798', 'curati

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
182eb872-395f-4248-97be-d481f08c9154,1,0,IM-1478_20220726_71D1S0VFBB.nwb,1a686739-cdee-4521-bca9-70913e0bf917,0,after metric curation
20fa485c-bdbb-456a-9e83-17a9bea8b2df,1,0,IM-1478_20220726_DP739VRJ5T.nwb,c04edfdd-9f5d-4a35-add4-3a86ab39f8f8,0,after metric curation
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,1,0,IM-1478_20220726_DVHTTNXJWV.nwb,30dface5-6a99-4f39-a2ba-ec01e0adf8a7,0,after metric curation
38635c94-2bea-493e-998d-4ffd420f070b,1,0,IM-1478_20220726_8OJINLDXEP.nwb,7d312a20-857b-454c-8db6-81d286c2cf73,0,after metric curation
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,1,0,IM-1478_20220726_NJKKUXXKTV.nwb,3171ce37-9986-41c0-90fc-210288271c4a,0,after metric curation
4a006737-9754-41de-8bf2-fd46caa68ec1,1,0,IM-1478_20220726_6ZVSC0FHRP.nwb,583e40c2-c60d-4b7d-b71a-f97d7f3573c5,0,after metric curation
4ca04e9c-3352-48e0-8beb-a07281c72a1c,1,0,IM-1478_20220726_5CBWNDCH4U.nwb,082e4736-3bb9-4fff-b571-b67f5eecafdc,0,after metric curation
59ce8c71-5946-4ad6-9824-a78a5492e5b4,1,0,IM-1478_20220726_PNY6XK301N.nwb,cc08f2b0-ee12-4a59-b1df-80e9168e3f40,0,after metric curation
6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf,1,0,IM-1478_20220726_JHTAUUSMDP.nwb,bda660f2-eee8-4259-8236-482f3dc6b83b,0,after metric curation
6e698671-85b9-4fcb-979d-1293e1660a90,1,0,IM-1478_20220726_3QV0TZXZNF.nwb,5ff48932-b258-4fee-97d1-372cbf5b9b63,0,after metric curation


Curation keys (after metric curation):
[{'sorting_id': '6e698671-85b9-4fcb-979d-1293e1660a90', 'curation_id': 1}, {'sorting_id': 'e2417f7f-b7f4-4698-949f-03582177a710', 'curation_id': 1}, {'sorting_id': 'cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4', 'curation_id': 1}, {'sorting_id': 'f294456f-c177-42b4-9895-dbcb98757337', 'curation_id': 1}, {'sorting_id': 'b7c8c657-05ef-4b15-9a0f-a00218d5163c', 'curation_id': 1}, {'sorting_id': '7617faba-e28c-4846-b04e-3dbdac4f4540', 'curation_id': 1}, {'sorting_id': '40f8ce6b-c9c2-4af4-acaf-da32a4f73b51', 'curation_id': 1}, {'sorting_id': 'c9da7594-c6ee-43b4-9a1e-7c9bc21906f4', 'curation_id': 1}, {'sorting_id': 'a28382e1-eb6d-4411-95e3-c8c7a6fe2798', 'curation_id': 1}, {'sorting_id': '6cebac9b-f53b-461f-b2c3-4bf33c6cfbcf', 'curation_id': 1}, {'sorting_id': '4a006737-9754-41de-8bf2-fd46caa68ec1', 'curation_id': 1}, {'sorting_id': 'bd374141-ba9f-48d0-a83d-282649e94b17', 'curation_id': 1}, {'sorting_id': '81076d23-cdf1-4881-a726-a3d2d44090f6', 'curation_id': 1}

In [21]:
# Combined output of both rounds of curation!
display(sgs.CurationV1() & (curation_key_list_round2 + curation_key_list))

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
182eb872-395f-4248-97be-d481f08c9154,0,-1,IM-1478_20220726_CAC88V34XP.nwb,36bbc865-b980-41b1-8003-23e3b3e25379,0,initial automatic curation
182eb872-395f-4248-97be-d481f08c9154,1,0,IM-1478_20220726_71D1S0VFBB.nwb,1a686739-cdee-4521-bca9-70913e0bf917,0,after metric curation
20fa485c-bdbb-456a-9e83-17a9bea8b2df,0,-1,IM-1478_20220726_2LQZTIRXZT.nwb,aadc3255-3b43-4dfe-afbf-7409479f90ad,0,initial automatic curation
20fa485c-bdbb-456a-9e83-17a9bea8b2df,1,0,IM-1478_20220726_DP739VRJ5T.nwb,c04edfdd-9f5d-4a35-add4-3a86ab39f8f8,0,after metric curation
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,0,-1,IM-1478_20220726_CX7QVUQ6AQ.nwb,1388a9b6-a219-450f-a9fc-99623551683f,0,initial automatic curation
373f4eb7-9411-41bf-ad85-16bbaccb3dfa,1,0,IM-1478_20220726_DVHTTNXJWV.nwb,30dface5-6a99-4f39-a2ba-ec01e0adf8a7,0,after metric curation
38635c94-2bea-493e-998d-4ffd420f070b,0,-1,IM-1478_20220726_EFAK43PR86.nwb,77be9e52-3d8d-47e4-a61d-1859b37fbfe9,0,initial automatic curation
38635c94-2bea-493e-998d-4ffd420f070b,1,0,IM-1478_20220726_8OJINLDXEP.nwb,7d312a20-857b-454c-8db6-81d286c2cf73,0,after metric curation
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,0,-1,IM-1478_20220726_URF84UP1Z3.nwb,3e33ecce-864d-4e11-b80e-1cbfdf16a5dd,0,initial automatic curation
40f8ce6b-c9c2-4af4-acaf-da32a4f73b51,1,0,IM-1478_20220726_NJKKUXXKTV.nwb,3171ce37-9986-41c0-90fc-210288271c4a,0,after metric curation


## For now, we skip manual curation.

## Downstream usage (Merge table)

Regardless of Curation method used, to make use of spikeorting results in downstream pipelines like Decoding, we will need to insert it into the `SpikeSortingOutput` merge table. 

In [22]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput

# Insert the second round curation results into the merge table
for key in curation_key_list_round2:

    # Check if this key has already been inserted
    merge_insert_key = (sgs.CurationV1 & key).fetch("KEY", as_dict=True)
    merge_entry = (SpikeSortingOutput() & merge_insert_key)

    # If it hasn't been inserted yet, insert it
    if len(merge_entry.fetch()) == 0:
        SpikeSortingOutput.insert(merge_insert_key, part_name="CurationV1")
    else:
        print(f"Entry for {merge_insert_key} already exists in SpikeSortingOutput")

Entry for [{'sorting_id': UUID('6e698671-85b9-4fcb-979d-1293e1660a90'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('e2417f7f-b7f4-4698-949f-03582177a710'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('cb94bfc9-13f4-4aee-8d48-f9c7c5b011f4'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('f294456f-c177-42b4-9895-dbcb98757337'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('b7c8c657-05ef-4b15-9a0f-a00218d5163c'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('7617faba-e28c-4846-b04e-3dbdac4f4540'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('40f8ce6b-c9c2-4af4-acaf-da32a4f73b51'), 'curation_id': 1}] already exists in SpikeSortingOutput
Entry for [{'sorting_id': UUID('c9da7594-c6ee-43b4-9a1e-7c9bc21906f4'), 'curation_id': 1}] already exist

Look at our entries in `SpikeSortingOutput`

In [23]:
# For now we just restrict on nwb file name and curation id
# We could also restrict on sorter, interval_list_name, etc
selection_key = {
    "nwb_file_name": nwb_file_name,
    'curation_id': 1
}
merge_ids = SpikeSortingOutput().get_restricted_merge_ids(key=selection_key, sources="v1", as_dict=True)

# View all of our entries in the table
display(SpikeSortingOutput() & merge_ids)

merge_id,source
06f09f86-93b3-1f74-bc32-31edf2e3b863,CurationV1
09360af7-7244-7cd4-466c-ff290366318c,CurationV1
0e99a76b-31bf-aa50-55cf-55193998dad6,CurationV1
1078ff7b-4a11-b3d6-b81f-12b41d41d52b,CurationV1
1bc939bc-9189-f91a-1337-528b46d6356b,CurationV1
2f8f8149-cd9d-56a8-3bd4-54662980c0c5,CurationV1
3a316ee5-54ef-d4d9-3872-52dda62c1915,CurationV1
4633a287-c7b4-ae6f-5f58-36e18d6f85a3,CurationV1
489014ae-90bc-fcec-007d-8ba4003e15c3,CurationV1
593c4557-9b78-60d7-795a-de3d009156fb,CurationV1


---------------------------------------


# Decode from sorted spikes

The elements we will need to decode with sorted spikes are:
- `PositionGroup`
- `SortedSpikesGroup`
- `DecodingParameters`
- `encoding_interval`
- `decoding_interval`


In [24]:
# from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
# import spyglass.spikesorting.v1 as sgs


# sorter_keys = {
#     "nwb_file_name": nwb_file_name,
#     "sorter": sorter,
#     "curation_id": 1,
# }
# # check the set of sorting we'll use
# (
#     sgs.SpikeSortingSelection & sorter_keys
# ) * SpikeSortingOutput.CurationV1 & sorter_keys

# # get the merge_ids for the selected sorting
# spikesorting_merge_ids = SpikeSortingOutput().get_restricted_merge_ids(sorter_keys, restrict_by_artifact=False)


In [25]:
# # create a new sorted spikes group
# sgs.SortedSpikesGroup().create_group(
#     group_name="test_group",
#     nwb_file_name=nwb_file_name,
#     keys=[
#         {"spikesorting_merge_id": merge_id}
#         for merge_id in spikesorting_merge_ids
#     ],
#     unit_filter_params_name=unit_filter_params_name,
# )
# # check the new group
# sgs.SortedSpikesGroup & {
#     "nwb_file_name": nwb_file_name,
#     "sorted_spikes_group_name": "test_group",
# }

## Model parameters

As before we can specify the model parameters. The only difference is that we will use the `ContFragSortedSpikesClassifier` instead of the `ContFragClusterlessClassifier`.